In [1]:
import sys
sys.path.append('..')
import importlib
import datagit.drift_evaluators_dry_run
importlib.reload(datagit.drift_evaluators_dry_run)
from datagit.drift_evaluators_dry_run import run_drift_evaluator
from datagit.drift_evaluators import auto_merge_drift
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token)
repo = os.getenv("REPO") or "gh_org/repo"

drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=auto_merge_drift)
drift

Commit message Drift: dbt-drift/metrics/monthly_bookings.csv

Drift detected and automatically merged.
Failed to parse drift summary: Expected object or value


Traceback (most recent call last):
  File "/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/datagit/drift_evaluators_dry_run.py", line 48, in run_drift_evaluator
    drift_summary = parse_drift_summary(commit_message)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/datagit/drift_evaluators.py", line 42, in parse_drift_summary
    added_rows=pd.read_json(parts[1]),
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/pandas/io/json/_json.py

{'should_alert': False,
 'message': 'Drift detected:\n- 🆕 0 addition\n- ♻️ 6 modifications\n- 🗑️ 0 deletion'}

In [2]:
import datagit.dataset_helpers
from datagit.drift_evaluators import DriftEvaluatorContext, DriftEvaluation
importlib.reload(datagit.dataset_helpers)
from datagit.dataset_helpers import get_addition_deletion_and_diff
import pandas as pd

def example_drift_evaluator(data_drift_context: DriftEvaluatorContext) -> DriftEvaluation:
    new_dataframe =  data_drift_context["after"]
    old_dataframe =  data_drift_context["before"]
    print("summary", data_drift_context["summary"]  )
    additions, deletions, diff= get_addition_deletion_and_diff(old_dataframe, new_dataframe, "unique_key")

    print("additions", additions)
    print("deletions", deletions)
    print("diff", diff)
    diff_column = diff.columns[0][0]
    diff = diff.apply(pd.to_numeric)  # Convert the values in the diff dataframe to numeric values
    
    print("computing the diff")
    monthly_booking_drift = diff[(diff_column, "other")] - diff[(diff_column, "self")]
    print("monthly_booking_drift", monthly_booking_drift)
    if(monthly_booking_drift > 1000).any():
        return {"should_alert": True, "message": "The monthly booking drift is too high"}
    return {"should_alert": False, "message": "Small drift"}

# no_drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=example_drift_evaluator)
# print("no_drift", no_drift)

large_drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="0aa423b04afa9f51c81a986e644862f5ae04a42f", drift_evaluator=example_drift_evaluator)
print("large_drift", large_drift)

Commit message DRIFT

Drift detected:
- 🆕 0 addition
- ♻️ 1 modification
- 🗑️ 6 deletions

Drift Summary:
Added Rows:
Empty DataFrame
Columns: [name, date, age]
Index: []
Deleted Rows:
Empty DataFrame
Columns: [name, date, age]
Index: []
Modified Rows Unique Keys:
Index(['2023-01-Charlie'], dtype='string', name='unique_key')
Modified Patterns:
         unique_keys column old_value new_value           pattern_id
0  [2023-01-Charlie]    age        36        35  4049105196195111928
Drift Summary Json Begin
Added Rows JSON:
{"name":{},"date":{},"age":{}}
Deleted Rows JSON:
{"name":{},"date":{},"age":{}}
Modified Rows Unique Keys List:
['2023-01-Charlie']
Modified Patterns Json:
{"unique_keys":{"0":["2023-01-Charlie"]},"column":{"0":"age"},"old_value":{"0":"36"},"new_value":{"0":"35"},"pattern_id":{"0":4049105196195111928}}
Drift Summary Json End
summary {'added_rows': Empty DataFrame
Columns: [name, date, age]
Index: [], 'deleted_rows': Empty DataFrame
Columns: [name, date, age]
Index: [],

In [3]:
import json
import pandas as pd 

coucou = pd.Index(['2023-01-Charlie'])